In [24]:
import pandas as pd
import os
from google.cloud import bigquery
from google.oauth2 import service_account
from pandas_gbq import to_gbq
import wget
from zipfile import ZipFile

### Url de dowload das Demonstrações Financeiras Contabeis(DFC)

In [25]:
url_cvm = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'

### Carregar Credencial do Google Big Query(GBQ)

In [26]:
#Carrega o arquivo json com a chave de autenticação do GBQ
credencial = service_account.Credentials.from_service_account_file(
    r'C:\Users\usuario\Documents\gbq.json',
    scopes=['https://www.googleapis.com/auth/bigquery']
)

### Criar lista com nome dos arquivos das DFCs

In [16]:
arquivos_zip = []

#Preencher a Lista para os anos de 2019 a 2023
for ano in range(2019, 2024):
    #Adiciona cada arquivo zip a lista criada
    arquivos_zip.append(f'dfp_cia_aberta_{ano}.zip')
arquivos_zip

['dfp_cia_aberta_2019.zip',
 'dfp_cia_aberta_2020.zip',
 'dfp_cia_aberta_2021.zip',
 'dfp_cia_aberta_2022.zip',
 'dfp_cia_aberta_2023.zip']

### Fazer dowload de todos os arquivos
• Verifica se já existe um arquivo com o nome e apaga e substitui pelo novo fazendo dowload novamente

In [17]:
for arquivo in arquivos_zip:
    #Caminho para salvar cada arquivo
    caminho_arquivo_zip = f'Demonstracoes_Zip/{arquivo}'
    #Caso já exista um arquivo com mesmo nome, é apagado
    if os.path.exists(caminho_arquivo_zip):
        os.remove(caminho_arquivo_zip)

    #Faz o dowload de cada arquivo zip de cada anos das DFC dentro da pasta Demonstracoes_Zip
    wget.download(url_cvm + arquivo, caminho_arquivo_zip)

### Extrair arquivos da pasta zipada

In [18]:
for arquivo in arquivos_zip:
    #Extrair os arquivos Zip das DFC nas pasta Demonstracoes
    ZipFile(f'Demonstracoes_Zip/{arquivo}', 'r').extractall('Demonstracoes')

### Carrega a lista dos arquivos do diretório Demonstracoes

In [19]:
#Diretorio do arquivo atual
diretorio_atual = os.getcwd()
caminho_diretorio = os.path.join(diretorio_atual, 'Demonstracoes')


arquivos_dfc = os.listdir('Demonstracoes/')

print(arquivos_dfc)

['dfp_cia_aberta_2019.csv', 'dfp_cia_aberta_2020.csv', 'dfp_cia_aberta_2021.csv', 'dfp_cia_aberta_2022.csv', 'dfp_cia_aberta_2023.csv', 'dfp_cia_aberta_BPA_con_2019.csv', 'dfp_cia_aberta_BPA_con_2020.csv', 'dfp_cia_aberta_BPA_con_2021.csv', 'dfp_cia_aberta_BPA_con_2022.csv', 'dfp_cia_aberta_BPA_con_2023.csv', 'dfp_cia_aberta_BPA_ind_2019.csv', 'dfp_cia_aberta_BPA_ind_2020.csv', 'dfp_cia_aberta_BPA_ind_2021.csv', 'dfp_cia_aberta_BPA_ind_2022.csv', 'dfp_cia_aberta_BPA_ind_2023.csv', 'dfp_cia_aberta_BPP_con_2019.csv', 'dfp_cia_aberta_BPP_con_2020.csv', 'dfp_cia_aberta_BPP_con_2021.csv', 'dfp_cia_aberta_BPP_con_2022.csv', 'dfp_cia_aberta_BPP_con_2023.csv', 'dfp_cia_aberta_BPP_ind_2019.csv', 'dfp_cia_aberta_BPP_ind_2020.csv', 'dfp_cia_aberta_BPP_ind_2021.csv', 'dfp_cia_aberta_BPP_ind_2022.csv', 'dfp_cia_aberta_BPP_ind_2023.csv', 'dfp_cia_aberta_DFC_MD_con_2019.csv', 'dfp_cia_aberta_DFC_MD_con_2020.csv', 'dfp_cia_aberta_DFC_MD_con_2021.csv', 'dfp_cia_aberta_DFC_MD_con_2022.csv', 'dfp_cia_abe

### Filtrar arquivos para carga no GBQ
É filtrado os arquivos que contém informações cadastrais, os arquivos com parecer de auditória e os arquivos das DFC individual, onde as informações são referentes apenas a própria empresa, desconsiderando outras empresas do grupo.

In [20]:
#Cria um DataFrame(DF) vazio
df = pd.DataFrame()

for arquivo in arquivos_dfc:
    #Filtrar arquivos a serem desconsiderados, arquivo de dados cadastrais sempre terá 23 digitos
    if 'parecer' in arquivo.lower() or len(arquivo) == 23 or 'ind' in arquivo.lower():
        pass
    else:
        #Passa o caminho de cada arquivo a ser análisado na pasta Demonstracoes
        caminho_arquivo = os.path.join(caminho_diretorio, arquivo)
        #Gera um DF de cada arquivo do diretório Demonstracoes
        df_arquivo = pd.read_csv(caminho_arquivo, sep=';', decimal=',', encoding='ISO-8859-1')
        #Adiciona cada arquivo dentro do DF vazio
        df = pd.concat([df, df_arquivo], ignore_index=True)



print(df.head())


             CNPJ_CIA    DT_REFER  VERSAO        DENOM_CIA  CD_CVM  \
0  00.000.000/0001-91  2019-12-31       2  BCO BRASIL S.A.    1023   
1  00.000.000/0001-91  2019-12-31       2  BCO BRASIL S.A.    1023   
2  00.000.000/0001-91  2019-12-31       2  BCO BRASIL S.A.    1023   
3  00.000.000/0001-91  2019-12-31       2  BCO BRASIL S.A.    1023   
4  00.000.000/0001-91  2019-12-31       2  BCO BRASIL S.A.    1023   

                                    GRUPO_DFP MOEDA ESCALA_MOEDA ORDEM_EXERC  \
0  DF Consolidado - Balanço Patrimonial Ativo  REAL          MIL   PENÚLTIMO   
1  DF Consolidado - Balanço Patrimonial Ativo  REAL          MIL      ÚLTIMO   
2  DF Consolidado - Balanço Patrimonial Ativo  REAL          MIL   PENÚLTIMO   
3  DF Consolidado - Balanço Patrimonial Ativo  REAL          MIL      ÚLTIMO   
4  DF Consolidado - Balanço Patrimonial Ativo  REAL          MIL   PENÚLTIMO   

  DT_FIM_EXERC CD_CONTA                       DS_CONTA               VL_CONTA  \
0   2018-12-31   

### Apagar dados da tabela raw_dfc
É executado consulta sql no GBQ para apagar os dados da tabela para não duplicar caso já tenha sido feita uma carga com os dados anteriormente

In [21]:
#Instanciar o cliente do GBQ
cliente = bigquery.Client(credentials=credencial, project=credencial.project_id)

#Query pra apagar os dados da tabela raw_dfc onde será carregado os dados das DFC
sql_apagar_dados = """ 
TRUNCATE TABLE `prjeto.dfc.raw_dfc` 
"""

#Executar a query de apagar os dados da tabela
query = cliente.query(sql_apagar_dados)




### Inserir dados dentro da tabela raw_dfc
• É realizado a inserção de todos os dados do DF consolidados das DFC dentro da tabela raw_dfc no GBQ

In [22]:
#Inserir os dados do DF consolidado das DFC na tabela raw_dfc do GBQ
df.to_gbq(
    destination_table='prjeto.dfc.raw_dfc',
    project_id='prjeto',
    if_exists='replace',
    credentials=credencial)

100%|██████████| 1/1 [00:00<?, ?it/s]


### Consulta para verificar se foram carregados os dados na tabela

In [27]:

#Query pra apagar os dados da tabela raw_dfc onde será carregado os dados das DFC
sql_selecionar_dados = """ 
SELECT * FROM `prjeto.dfc.raw_dfc` LIMIT 10
"""

#Executar a query de apagar os dados da tabela
query = cliente.query(sql_selecionar_dados)

# Coletar os resultados
results = query.result()

# Exibir os resultados
for row in results:
    print(row)


Row(('11.284.204/0001-18', '2019-12-31', 5, 'YOU INC INCORPORADORA E PARTICIPAÇÃO S.A', 23604, 'DF Consolidado - Balanço Patrimonial Ativo', 'REAL', 'MIL', 'PENÚLTIMO', '2018-12-31', '1', 'Ativo Total', '805724.0000000000', 'S', None, None), {'CNPJ_CIA': 0, 'DT_REFER': 1, 'VERSAO': 2, 'DENOM_CIA': 3, 'CD_CVM': 4, 'GRUPO_DFP': 5, 'MOEDA': 6, 'ESCALA_MOEDA': 7, 'ORDEM_EXERC': 8, 'DT_FIM_EXERC': 9, 'CD_CONTA': 10, 'DS_CONTA': 11, 'VL_CONTA': 12, 'ST_CONTA_FIXA': 13, 'DT_INI_EXERC': 14, 'COLUNA_DF': 15})
Row(('11.284.204/0001-18', '2019-12-31', 5, 'YOU INC INCORPORADORA E PARTICIPAÇÃO S.A', 23604, 'DF Consolidado - Balanço Patrimonial Ativo', 'REAL', 'MIL', 'PENÚLTIMO', '2018-12-31', '1.01', 'Ativo Circulante', '740068.0000000000', 'S', None, None), {'CNPJ_CIA': 0, 'DT_REFER': 1, 'VERSAO': 2, 'DENOM_CIA': 3, 'CD_CVM': 4, 'GRUPO_DFP': 5, 'MOEDA': 6, 'ESCALA_MOEDA': 7, 'ORDEM_EXERC': 8, 'DT_FIM_EXERC': 9, 'CD_CONTA': 10, 'DS_CONTA': 11, 'VL_CONTA': 12, 'ST_CONTA_FIXA': 13, 'DT_INI_EXERC': 14